In [1]:
# 1) dates should be formatted in excel to yyyy-MM-dd
# 2) data input as excel format

In [2]:
%reset -f

In [3]:
import pandas as pd
import numpy as np
import os

from datetime import datetime, timedelta

In [4]:
filename = "//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Input/current_bsfs_barnn.xlsx"
x1 = pd.ExcelFile(filename) # check this out!

In [5]:
# res = len(x1.sheet_names)
sheetList = x1.sheet_names
sheetList.sort()
# sheetList

In [6]:
df = pd.read_excel(filename,sheet_name='Current Qtr',header=0)
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('#', 'num')
df.columns = df.columns.str.replace('%', 'PER')

In [7]:
filename2 = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/Incentive Requirments/BSFS_BARNN_Incentive.txt'
df_tierMap = pd.read_csv(filename2, sep = '\t')
# df_tierMap

In [8]:
df = df.assign(QTDsum=0,CurrentTier=0,CreateDate=0)

df['QTDsum'] = df.MB_Prime + df.MB_2nd
df.loc[df['QTDsum'] < 225, 'CurrentTier'] = 0
df.loc[df['QTDsum'] >= 225, 'CurrentTier'] = 1

In [9]:
tup_now = datetime.now().timetuple()
currentYear = datetime.now().year

In [10]:
ts_currentData = os.path.getctime(filename)
dt_currentData = datetime.fromtimestamp(ts_currentData)
tup_currentData = dt_currentData.timetuple()
# print(tup_currentData)
df.CreateDate = str(tup_currentData.tm_year) + '-' + str(tup_currentData.tm_mon) + '-' + str(tup_currentData.tm_mday)

ts_incentive = os.path.getctime(filename2)
dt_incentive = datetime.fromtimestamp(ts_incentive)
# print(dt_currentData)

In [11]:
calculatedData = pd.DataFrame([])
for ii in range(df.shape[0]):

    singleCusty = df.iloc[[ii]]
    singleCusty.reset_index(inplace=True)

    if  tup_currentData.tm_year == currentYear:

        if (tup_currentData.tm_mon == 1) | (tup_currentData.tm_mon == 2) | (tup_currentData.tm_mon == 3):
            if ((tup_currentData.tm_year % 400 == 0) | (tup_currentData.tm_year % 4 == 0)) & (tup_currentData.tm_year % 100 != 0):
                quarter = 1
                daysINqtr = 91
                print('Leap year')
            else:
                quarter = 1
                daysINqtr = 90           
        elif (tup_currentData.tm_mon == 4) | (tup_currentData.tm_mon == 5) | (tup_currentData.tm_mon == 6):
            quarter = 2
            daysINqtr = 91
            subtractDays = 90          
        elif (tup_currentData.tm_mon == 7) | (tup_currentData.tm_mon == 8) | (tup_currentData.tm_mon == 9):
            quarter = 3
            daysINqtr = 92
            subtractDays = 90 + 91
        else:
            quarter = 4
            daysINqtr = 92
            subtractDays = 90 + 91 + 92
        
        numDays = tup_currentData.tm_yday - subtractDays # get number of days

        estimatedTires = np.floor((singleCusty.QTDsum / (numDays / daysINqtr))).values

        if   (estimatedTires >= (df_tierMap.iloc[1,4])) & (singleCusty.Percent_MB[0] >= 0.75):
            tierType = 1
        else:
            tierType = 0
        
        if tierType < 1:
            tierUP = tierType + 1
        else:
            tierUP = 1

        if singleCusty.CurrentTier.values == 1:
            currentBSdollars = df_tierMap.iloc[1,1] * singleCusty.BS_Prime
            currentFSdollars = df_tierMap.iloc[1,2] * singleCusty.FS_Prime
            currentFZdollars = df_tierMap.iloc[1,3] * singleCusty.FZ_Prime
        else:
            currentBSdollars = 0
            currentFSdollars = 0
            currentFZdollars = 0
        Q1_current_rebate = currentBSdollars + currentFSdollars + currentFZdollars
        
        # floor to underestimate estimated tires
        bridgeStone_est_tires = np.floor((singleCusty.BS_Prime / (numDays / daysINqtr)).values)
        bridgeStone_est_tires_rebate = bridgeStone_est_tires * df_tierMap.iloc[tierType,1]
        bridgeStone_prop = singleCusty.BS_Prime / singleCusty.QTDsum
        # ceil to make sure the customer surpasses the minimal 1up tire threshold
        bridgeStone_1up_tires = np.ceil((bridgeStone_prop * df_tierMap.iloc[tierUP,4]).values)
        bridgeStone_1up_tires_rebate = bridgeStone_1up_tires * df_tierMap.iloc[tierUP,1]
   
        fireStone_est_tires = np.floor((singleCusty.FS_Prime / (numDays / daysINqtr)).values)
        fireStone_est_tires_rebate = fireStone_est_tires * df_tierMap.iloc[tierType,2]
        fireStone_prop = singleCusty.FS_Prime / singleCusty.QTDsum
        fireStone_1up_tires = np.ceil((fireStone_prop * df_tierMap.iloc[tierUP,4]).values)
        fireStone_1up_tires_rebate = fireStone_1up_tires * df_tierMap.iloc[tierUP,2]
        
        fuzion_est_tires = np.floor((singleCusty.FZ_Prime / (numDays / daysINqtr)).values)
        fuzion_est_tires_rebate = fuzion_est_tires * df_tierMap.iloc[tierType,3]
        fuzion_prop = singleCusty.FZ_Prime / singleCusty.QTDsum
        fuzion_1up_tires = np.ceil((fuzion_prop * df_tierMap.iloc[tierUP,4]).values)
        fuzion_1up_tires_rebate = fuzion_1up_tires * df_tierMap.iloc[tierUP,3]
     
        total_est_dollars = bridgeStone_est_tires_rebate + fireStone_est_tires_rebate + fuzion_est_tires_rebate
        total_1up_dollars = bridgeStone_1up_tires_rebate + fireStone_1up_tires_rebate + fuzion_1up_tires_rebate
        total_1up_tires = bridgeStone_1up_tires + fireStone_1up_tires
        MB_est_tires = bridgeStone_est_tires + fireStone_est_tires
        Q1_MB_total = singleCusty.MB_Prime + singleCusty.MB_2nd 
        
        if estimatedTires > total_1up_tires:
            bridgeStone_1up_tires = bridgeStone_est_tires
            fireStone_1up_tires = fireStone_est_tires
            fuzion_1up_tires = fuzion_est_tires
            total_1up_tires = estimatedTires
            
            bridgeStone_1up_tires_rebate = bridgeStone_est_tires_rebate
            fireStone_1up_tires_rebate = fireStone_est_tires_rebate
            fuzion_1up_tires_rebate = fuzion_est_tires_rebate
            total_1up_dollars = total_est_dollars

        range_dollars_ll = ((total_1up_tires * df_tierMap.iloc[tierUP,2]))
        range_dollars_ul = ((total_1up_tires * df_tierMap.iloc[tierUP,1]))
        
        tempData = pd.DataFrame(
            {'estimated_tier':tierType, 'currentBSdollars':currentBSdollars, 'currentFSdollars':currentFSdollars, 'currentFZdollars':currentFZdollars, 'current_tires':Q1_MB_total, 'current_total_rebate':Q1_current_rebate,
             'bridgeStone_est_tires':bridgeStone_est_tires, 'fireStone_est_tires':fireStone_est_tires, 'fuzion_est_tires':fuzion_est_tires, 'current_est_tires':MB_est_tires,
             'bridgeStone_est_rebate':bridgeStone_est_tires_rebate, 'fireStone_est_rebate':fireStone_est_tires_rebate, 'fuzion_est_rebate':fuzion_est_tires_rebate, 'total_est_rebate':total_est_dollars,
             'Up_tier':tierUP,
             'bridgeStone_up_tires':bridgeStone_1up_tires, 'fireStone_up_tires':fireStone_1up_tires, 'fuzion_up_tires':fuzion_1up_tires,
             'bridgeStone_up_rebate':bridgeStone_1up_tires_rebate, 'fireStone_up_rebate':fireStone_1up_tires_rebate, 'fuzion_up_rebate':fuzion_1up_tires_rebate, 'total_up_rebate':total_1up_dollars,
             'range_lower':range_dollars_ll, 'range_upper':range_dollars_ul,'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
    else:
        range_dollars_ll = (df_tierMap.iloc[1,3] * df_tierMap.iloc[1,4])
        range_dollars_ul = (df_tierMap.iloc[1,1] * df_tierMap.iloc[1,4]) 
        tempData = pd.DataFrame(
            {'estimated_tier':0,  'currentBSdollars':0, 'currentFSdollars':0, 'currentFZdollars':0, 'current_tires':0, 'current_total_rebate':0,
             'bridgeStone_est_tires':0, 'fireStone_est_tires':0, 'fuzion_est_tires':0, 'current_est_tires':0,
             'bridgeStone_est_rebate':0, 'fireStone_est_rebate':0, 'fuzion_est_rebate':0, 'total_est_rebate':0,
             'Up_tier':1,
             'bridgeStone_up_tires':0, 'fireStone_up_tires':0, 'fuzion_up_tires':0,
             'bridgeStone_up_rebate':0, 'fireStone_up_rebate':0, 'fuzion_up_rebate':0, 'total_up_rebate':0,
             'range_lower':[range_dollars_ll], 'range_upper':[range_dollars_ul],'period': quarter, 'grain': 'Quarter'})
        calculatedData = calculatedData.append(tempData)
###########################################################################################################################        
calculatedData = calculatedData.reset_index(drop=True)
finalData = pd.concat([calculatedData,df], axis=1)

C:\dev\2019_04_29_Anaconda\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: invalid value encountered in greater


In [12]:
finalData = finalData.replace([np.inf, -np.inf], np.nan).fillna(0)
# finalData.head()

In [13]:
path = '//usoil.local/Production/USV/Apps/PowerBI/US AutoForce/Sales Reports/CMP/python_csv_output/'
writeFilename = path + 'CMP_BSFS_BARNN.csv'
finalData.to_csv(writeFilename, index=False)
writeFilename_archive = path + 'csv_archive/' + str(tup_now.tm_year) + '_' + str(tup_now.tm_mon).zfill(2) \
                        + '_' + str(tup_now.tm_mday).zfill(2) + '_' + 'CMP_BSFS_BARNN_' \
                        + str(tup_now.tm_hour).zfill(2) + str(tup_now.tm_min).zfill(2) + str(tup_now.tm_sec).zfill(2) \
                        + '.csv'
finalData.to_csv(writeFilename_archive, index=False)